In [ ]:
pip install finterstellar

In [ ]:
import pandas as pd
import numpy as np
import finterstellar as fs
import plotly.express as px

In [ ]:
cd = ['SPY', 'SCHX']

시세 읽어오기

In [ ]:
df1 = fs.get_price(cd[0], start_date='2022-01-01')
df2 = fs.get_price(cd[1], start_date='2022-01-01')
df_ = pd.concat([df1, df2], axis=1)
df_.fillna(method='ffill', inplace=True)
px.line(df_)

In [ ]:
df = (df_.pct_change()+1).cumprod()    # 누적곱 수익률
df.dropna(inplace=True)
px.line(df)

In [ ]:
df2 = np.log(df)    # 로그 수익률
px.line(df2)

상관관계 분석

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.stattools import coint

In [ ]:
# correlation
df[cd[0]].corr(df[cd[1]])

In [ ]:
# cointegration
coint_t, p_value, crit_value = coint(df[cd[0]], df[cd[1]])
print('{:.4f}'.format(coint_t), '{:.4f}'.format(p_value), crit_value)

상관관계가 있다면, 추세를 파악

In [ ]:
def regression(x, y):
    # x, y를 열백터로 변환
    x = np.array(x).reshape(-1, 1)
    y = np.array(y).reshape(-1, 1)
    # Linear Regression
    model = LinearRegression()
    model.fit(x, y)
    result = {'Slope':model.coef_[0,0], 'Intercept':model.intercept_[0], 'R2':model.score(x, y) }
    return(result)

In [ ]:
regr = regression(df[cd[0]], df[cd[1]])
regr

추세선을 기준으로 x 주가를 이용해 y 주가를 계산

In [ ]:
df[cd[1]+' expected'] = df[cd[0]] * regr['Slope'] + regr['Intercept']
# y주가 기대값은 추세선 상에 위치
df[cd[1]+' spread'] = df[cd[1]] - df[cd[1]+' expected']
# 스프레드 = y주가 - 기대값
df.head()

역치값 설정

In [ ]:
# trading strategy determinants
threshold = .005   # percent 값으로 오차허용 구간을 정의

트레이딩북 생성

In [ ]:
book = pd.DataFrame()    # 빈 트레이딩북(데이터프레임) 생성
book[cd] = df[cd]    # 시세 복사
book['trade'] = ''    # 매매전략 입력을 위한 trade 컬럼 생성
for c in cd:    # 종목쌍 내의 각 종목에 대해
    book['p '+c] = ''     # 포지션 입력을 위한 p 종목코드 컬럼 생성

In [ ]:
book

일별 매매내역

In [ ]:
for i in df.index:
    thd = float( threshold * df.loc[i, cd[1]] )    # 오차구간 계산
    if df.loc[i, cd[1]+' spread'] > thd:     # if 스프레드 > 오차구간 :
        book.loc[i, 'trade'] = 'buy '+cd[0]      # trade : buy 종목X
    elif threshold >= df.loc[i, cd[1]+' spread'] and df.loc[i, cd[1]+' spread'] >= 0:
    # elif 오차구간 >= 스프레드 & 스프레드 >= 0 :
        book.loc[i, 'trade'] = ''     # trade : clear
    elif 0 > df.loc[i, cd[1]+' spread'] and df.loc[i, cd[1]+' spread'] >= -thd:
    # elif 0 >= 스프레드 & 스프레드 >= -오차구간 :
        book.loc[i, 'trade'] = ''     # trade : clear
    elif -threshold > df.loc[i, cd[1]+' spread']:
    # elif -오차구간 > 스프레드 :
        book.loc[i, 'trade'] = 'buy '+cd[1]     # trade : buy 종목Y

In [ ]:
book.head()

포지션 기록

In [ ]:
for c in cd:
    status = ''
    for i in book.index:
        if book.loc[i, 'trade'] == 'buy '+c:     # 현재 buy 일때,
            if book.shift(1).loc[i, 'trade'] == 'buy '+c:   # 전일도 buy 였으면
                status = 'll'     # 롱>롱
            elif book.shift(1).loc[i, 'trade'] == '':    # 전일에 zero 였으면
                status = 'zl'     # zero>롱
            elif book.shift(1).loc[i, 'trade'] == 'sell '+c:    # 전일에 sell 였으면
                status = 'sl'     # 숏>롱
            else:    # 그밖의 경우
                status = 'zl'    # zero>롱
        elif book.loc[i, 'trade'] == 'sell '+c:
            if book.shift(1).loc[i, 'trade'] == 'buy '+c:
                status = 'ls'
            elif book.shift(1).loc[i, 'trade'] == '':
                status = 'zs'
            elif book.shift(1).loc[i, 'trade'] == 'sell '+c:
                status = 'ss'
            else:
                status = 'zs'
        elif book.loc[i, 'trade'] == '':
            if book.shift(1).loc[i, 'trade'] == 'buy '+c:
                status = 'lz'
            elif book.shift(1).loc[i, 'trade'] == '':
                status = 'zz'
            elif book.shift(1).loc[i, 'trade'] == 'sell '+c:
                status = 'sz'
            else:
                status = 'zz'
        else:
            status = 'zz'
        book.loc[i, 'p '+c] = status

In [ ]:
book[book['trade']!='']

수익률 계산

In [ ]:
rtn = 1.0
book['return'] = 1
for c in cd:     # 종목별로 순환
    buy = 0.0
    sell = 0.0
    for i in book.index:     # 일자별로 순환

        if book.loc[i, 'p '+c] == 'zl' or book.loc[i, 'p '+c] == 'sl' :     # long 진입
            buy = book.loc[i, c]    # 매수 가격 확정
            print(i.date(), 'long '+c, buy)
        elif book.loc[i, 'p '+c] == 'lz' or book.loc[i, 'p '+c] == 'ls' :     # long 청산
            sell = book.loc[i, c]    # 매도 가격 확정
            # 손익 계산
            rtn = sell / buy
            # 손익 = (매도가-매수가)/매수가 + 1 , 100원 투자해서 10원 벌면 손익은 1.10
            book.loc[i, 'return'] = rtn    # 트레이딩북에 손익 기록
            print(i.date(), 'long '+c, buy, ' | unwind long '+c, sell, ' | return:', round(rtn, 4))

        elif book.loc[i, 'p '+c] == 'zs' or book.loc[i, 'p '+c] == 'ls' :     # short 진입
            sell = book.loc[i, c]    # 공매도 가격 확정
            print(i.date(), 'short '+c, sell)
        elif book.loc[i, 'p '+c] == 'sz' or book.loc[i, 'p '+c] == 'sl' :     # short 청산
            buy = book.loc[i, c]    # 숏커버 가격 확정
            # 손익 계산
            rtn = buy / sell
            book.loc[i, 'return'] = rtn
            print(i.date(), 'short '+c, sell, ' | unwind short '+c, buy, ' | return:', round(rtn, 4))

    if book.loc[i, 'trade'] == '' and book.loc[i, 'p '+c] == '':     # zero position
        buy = 0.0
        sell = 0.0

acc_rtn = 1.0
for i in book.index:
    rtn = book.loc[i, 'return']
    acc_rtn = acc_rtn * rtn
    book.loc[i, 'acc return'] = acc_rtn

print ('Accunulated return :', round(acc_rtn, 4))

벤치마크 수익률

In [ ]:
n = len(cd)    # 몇 종목인지?
rtn = dict()    # 각 종목의 수익률을 담을 딕셔너리 선언
bm_rtn = float()    # 벤치마크 수익률을 담을 변수 선언
for c in cd:    # 종목마다 돌면서
    rtn[c] = round ( book[c].iloc[-1] / book[c].iloc[0] , 4)
    # 종목별 벤치마크 수익률 = (최종가격 - 최초가격) / 최초가격
    bm_rtn += rtn[c]/n   # 각 종목 수익률을 종목 개수로 나누고 합쳐준다
print('BM return:', round(bm_rtn, 4) )
print(rtn)

초과수익률 %

In [ ]:
exs_rtn =  acc_rtn - bm_rtn
print('Excess return:', round(exs_rtn, 4) )

그래프 모듈 호출

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def trade_result_chart():
    df['pos0'] = 0    # 포지션 기록용 컬럼 추가, 디폴트로 0 세팅
    df['pos1'] = 0
    df.loc[book['p '+cd[0]].str[1]=='l', 'pos0'] = 1    # long 포지션은 1 기록
    df.loc[book['p '+cd[1]].str[1]=='l', 'pos1'] = 1    # long 포지션은 1 기록

    df['acc_rtn'] = book['acc return']

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df.index, y=df[cd[0]], name=cd[0], line={'color':'rgba(200,20,60,1)'}),
        secondary_y=True,
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df[cd[1]], name=cd[1], line={'color':'rgba(30,144,255,1)'}),
        secondary_y=True,
    )

    fig.add_trace(
        go.Scatter(x=df.index, y=df['pos0'], name=cd[0], fill='tozeroy', line={'color':'rgba(200,20,60,.5)'}, fillcolor='rgba(200,20,60,.3)', yaxis='y3'),
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df['pos1'], name=cd[1], fill='tozeroy', line={'color':'rgba(30,144,255,.5)'}, fillcolor='rgba(30,144,255,.3)', yaxis='y3'),
    )

    fig.add_trace(
        go.Scatter(x=df.index, y=df['acc_rtn'], name='Return', line={'color':'white'}),
    )

    # Set y-axes titles
    fig.update_yaxes(secondary_y=True)

    fig.update_layout(
        template='plotly_dark',
        margin=dict(l=10, r=10, t=10, b=10),
        legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01),
        yaxis2=dict(title='BM return', anchor='x', overlaying='y', side='right', tickmode='sync',),
        yaxis3=dict(title='Position', anchor='free', overlaying='y', side='left', position=.1, visible=False),
    )
    fig.show()

In [ ]:
trade_result_chart()